# RESNET50 model for MNIST Dataset

---


![](https://cdn-images-1.medium.com/max/1600/1*hEU7S-EiVqcmtAlj6kgfRA.png) 

## Architecture
---

![](https://neurohive.io/wp-content/uploads/2019/01/resnet-architectures-34-101.png)

## Code
---


In [0]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential , Model
from keras.layers import Input ,  Dense, Dropout, Flatten , Activation , ZeroPadding2D
from keras.layers import Conv2D, MaxPooling2D  , Add , AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras.regularizers import l2

batch_size = 256
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)




##############################################################################################


def ident_block(input_X , filters , kernels):

  K1 , K2 , K3 = kernels
  F1 , F2 , F3 = filters
  x_shortcut = input_X
  
  conv1 = Conv2D(filters= F1 , kernel_size= (K1 , K1), padding = 'valid')(input_X)
  bn1 = BatchNormalization()(conv1)
  act1 = Activation('relu')(bn1)
  
  conv2 = Conv2D(filters= F2 , kernel_size= (K2 , K2) , padding='same')(act1)
  bn2 = BatchNormalization()(conv2)
  act2 = Activation('relu')(bn2)
  
  conv3 = Conv2D(filters= F3 , kernel_size= (K3,K3), padding = 'valid')(act2)
  bn3 = BatchNormalization()(conv3)
  
  X_add = Add()([input_X , bn3])
  act3 = Activation('relu')(X_add)

  return act3





def conv_block(input_X , filters , kernels):
  
  
  K1 , K2 , K3 = kernels
  F1 , F2 , F3 = filters
  x_shortcut = input_X
  
  conv1 = Conv2D(filters= F1 , kernel_size = (K1,K1))(input_X)
  bn1 = BatchNormalization()(conv1)
  act = Activation('relu')(bn1)
  
  conv2 = Conv2D(filters= F2 , kernel_size= (K2,K2) , padding= 'same')(act)
  bn2 = BatchNormalization()(conv2)
  act2 = Activation('relu')(bn2)
   
  conv3 = Conv2D(filters= F3 , kernel_size= (K3 , K3) , padding= 'valid')(act2)
  bn3 = BatchNormalization()(conv3)
   
  shcut_conv1 = Conv2D(filters= F3 , kernel_size= (1,1))(x_shortcut)
  shcut_bn1 = BatchNormalization()(shcut_conv1)
  
  X_add = Add()([shcut_bn1 , bn3])
  act3 = Activation('relu')(X_add)
  
  return act3





  
# Input
X_in = Input(input_shape)

# Zero Padding
X = ZeroPadding2D((3,3))(X_in)

# Stage 1
# CONV -> BATCHNORM -> RELU -> POOL
conv_1 = Conv2D(filters = 64 , kernel_size= (7,7) ,strides= 2)(X)
batch_norm_1 = BatchNormalization()(conv_1)
relu_1 = Activation('relu')(batch_norm_1)
max_pool_1 = MaxPooling2D(pool_size=(3,3) , strides= 2)(relu_1)

# Stage 2
# CONV BLOCK -> ID BLOCK
conv_2 = conv_block(max_pool_1 , (64,64,256) , (1,3,1))
ident_2 = ident_block(conv_2 , (64,64,256) , (1,3,1))
ident_3 = ident_block(ident_2 , (64,64,256) , (1,3,1))

# Stage 3
conv_3 = conv_block(ident_3 , (128,128,512) , (1,3,1))
ident_4 = ident_block(conv_3 , (128,128,512) , (1,3,1))
ident_5 = ident_block(ident_4 , (128,128,512) , (1,3,1))
ident_6 = ident_block(ident_5 , (128,128,512) , (1,3,1))

# Stage 4
conv_4 = conv_block(ident_6 , (256 , 256 , 1024) , (1,3,1))
ident_7 = ident_block(conv_4 , (256 , 256 , 1024) , (1,3,1))
ident_8 = ident_block(ident_7 , (256 , 256 , 1024) , (1,3,1))
ident_9 = ident_block(ident_8 , (256 , 256 , 1024) , (1,3,1))
ident_10 = ident_block(ident_9 , (256 , 256 , 1024) , (1,3,1))
ident_11 = ident_block(ident_10 , (256 , 256 , 1024) , (1,3,1))


# Stage 5
conv_5 = conv_block(ident_11 , (512 , 512 , 2048) , (1,3,1))
ident_12 = ident_block(conv_5 , (512 , 512 , 2048) , (1,3,1))
ident_13= ident_block(ident_12 , (512 , 512 , 2048) , (1,3,1))


# Average Pool
avg_pool =  AveragePooling2D((2,2))(ident_13)

# FC
flatten = Flatten()(avg_pool)
dense = Dense(num_classes , activation= 'softmax' )(flatten)

# Model
model = Model(inputs = X_in , outputs = dense )

############################################################################################




model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])